In [1]:
%pwd

'D:\\Work\\notebook\\metricstudy\\notebook'

In [2]:
BTC_ETH = "BTC_ETH"

poloniex_url_template = "https://poloniex.com/public?command=returnTradeHistory&currencyPair=%s&start=%s&end=%s"


In [3]:
#currency_pair: currency pair expression which is supported in poloniex, to: unixtime value
def get_url(currency_pair, end_unix):
    SIXMONTHS = 60*60*24*30*6
    ONEDAY = 60*60*24
    ONWWEEK = 7*ONEDAY
    
    date_range = ONEDAY
    start_unix = end_unix - date_range
    url = poloniex_url_template%(currency_pair, start_unix, end_unix)
    print (url)
    return url

In [4]:
import time
import datetime

def now():
    #now = int(time.gmtime())
    now = datetime.datetime.utcnow()
    now = int(time.mktime(now.timetuple()))
    #print(type(now))
    print (now)
    return now
#1493047494
now()

1493363946


1493363946

In [5]:
import datetime

def get_gmttime(lastdate):
    #print(datetime.datetime.strptime(lastdate, "%Y-%m-%d %H:%M:%S"))
    endtime = int(datetime.datetime.strptime(lastdate, "%Y-%m-%d %H:%M:%S").timestamp())+(9*60*60)
    #print(endtime)
    return endtime


In [6]:
import requests

def get_bth_eth(lasttime):
    url = get_url(BTC_ETH, lasttime)
    r = requests.get(url)
    js = r.json()
    
    lastdate = js[-1]['date']
    endtime = get_gmttime(lastdate)

    print ("downloaded %d items, by %s"%(len(js), lastdate))
    
    return (js, endtime)

#print(js[:2])
#print(js[-2:])

In [7]:
def get_batch_list(total_size, batch_size=100):
    total = [a for a in range(0, total_size)]
    #print (total)
    
    res = list()
    while len(total) > 0:
        bs = min(batch_size, len(total))
        #print (bs)
        a = total[:bs]
        res.append(a)
        if bs < len(total):
            b = total[bs:]
            total = b
        else:
            total = []
            break
    return res
    

In [8]:
import MySQLdb

In [9]:
conn = MySQLdb.connect(db='poloniex', user='polo', passwd='polo')
curr = conn.cursor()

In [10]:
def execute_query(query):
    try:
        curr.execute(query)
    except (MySQLdb.Error, MySQLdb.Warning) as e:
        print ("MySQL Error [%d]: %s" % (e.args[0], e.args[1]))
    except IndexError:
        print ("MySQL Error: %s" % str(e))

def get_db_size():
    curr.execute("SELECT COUNT(*) FROM btc_eth")
    (number_of_rows,)=curr.fetchone()
    return number_of_rows

In [11]:
#make insert statement
def add_data(trade_json):
    batch_size = 1000
    batch = get_batch_list(len(trade_json), batch_size)

    for bs in batch:
        query = "insert into btc_eth values "

        first = True
        for b in bs:
            item = trade_json[b]
            #print (item)

            value = "(%d,%d,%s,%s,%s,'%s','%s')"%(item['globalTradeID'], item['tradeID'], item['total'], item['rate'], item['amount'], item['type'], item['date'])

            if first:
                first = False
            else:
                query += ','

            query += value
        #print ("processing %d"%(len(bs)))
        #print (query)
        
        execute_query(query)
        #print ("done@@")
    
    #print ("begin commit")
    if len(batch) > 0:
        conn.commit()
    #print ("end commit")
    
    print ("trade data count = %d"%(get_db_size()))
    

In [12]:
import time
import datetime

starttime = "2016-06-22 07:31:22"
starttime = "2016-01-11 02:11:23"
start_timestamp = int(time.mktime(datetime.datetime.strptime(starttime, "%Y-%m-%d %H:%M:%S").timetuple()))
print (start_timestamp)
    

1452445883


In [13]:
#lasttime = now()
lasttime = start_timestamp
prev_lasttime = 0
while prev_lasttime != lasttime:
    prev_lasttime = lasttime
    (js_result, lasttime) = get_bth_eth(lasttime)
    print (lasttime)
    #print js_result
    #continue
        
    if len(js_result) == 0:
        break
    
    #add trade data to db
    add_data(js_result)
    

https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1452359483&end=1452445883
downloaded 6332 items, by 2016-01-09 17:15:19
1452359719
trade data count = 20944938
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1452273319&end=1452359719
downloaded 4800 items, by 2016-01-08 17:16:29
1452273389
MySQL Error [1062]: Duplicate entry '13402628' for key 'PRIMARY'
trade data count = 20948738
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1452186989&end=1452273389
downloaded 5579 items, by 2016-01-07 17:17:25
1452187045
MySQL Error [1062]: Duplicate entry '13370944' for key 'PRIMARY'
trade data count = 20953317
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1452100645&end=1452187045
downloaded 6241 items, by 2016-01-06 17:17:48
1452100668
MySQL Error [1062]: Duplicate entry '13342535' for key 'PRIMARY'
trade data count = 20958558
https://poloniex.com/public?comma

In [35]:
len(js_result) 

30452